In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import os
from scipy.stats import mode

In [38]:
# Load the model
model = keras.models.load_model('../fitmotion_model.keras')
labels = ['dws', 'jog', 'sit', 'std', 'ups', 'wlk']

c:\ProgramData\miniconda3\envs\tf_testing\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
# opening scaling factor file to read the dictionary
with open('../constants.txt', 'r') as f:
    lines = f.readlines()
    scaling_factors = eval(lines[0])

scaling_factors

{'Attitude Roll': 0.1142376591444776,
 'Attitude Pitch': 6.743238035682087,
 'Attitude Azimuth': -0.41661325633781365,
 'Gravity X': -0.40567461707781854,
 'Gravity Y': -0.09295193616982254,
 'Gravity Z': -0.10113856668202935,
 'Rotation Rate X': -1.4943574681964766,
 'Rotation Rate Y': -1.2789613806297648,
 'Rotation Rate Z': -1.6287858288885937,
 'User Acceleration X': -0.0263404192567685,
 'User Acceleration Y': 0.11506888215007838,
 'User Acceleration Z': 0.14220148004175853}

In [40]:
# Sequnce Generator
WINDOW_LENGTH = 150
STRIDE_LENGTH = 10
NUM_CLASSES = 6
NUM_FEATURES = 12
BATCH_SIZE = 100
EPOCHS_SIZE = 10

def sequence_generator(x, length, stride):
    seq_x = []
    data_length = len(x)

    for i in range(0, data_length - length + 1, stride):
        input_sequence = x.iloc[i : i + length]
        seq_x.append(input_sequence)
    return np.array(seq_x)

In [41]:
# load multiple data
data_path = '../data/android_data_cleaned/'

accuracy = {}

for dir in os.listdir(data_path):
    # create key according to each directory name
    accuracy[dir] = 0

    # predicting the data
    for file in os.listdir(data_path + dir):
        if file.endswith('.csv'):
            df = pd.read_csv(data_path + dir + '/' + file)
            df = df.drop(['Unnamed: 0'], axis=1) if 'Unnamed: 0' in df.columns else df
            df = df.drop(['id'], axis=1) if 'id' in df.columns else df

            print(data_path + dir + '/' + file)
            
            # Multiply data with constants
            for i in range(len(df.columns)):
                df.iloc[:, i] = df.iloc[:, i] * scaling_factors[df.columns[i]]
            
            # create sequences
            tx = sequence_generator(df, WINDOW_LENGTH, STRIDE_LENGTH) 

            # predict the sequences
            y_pred = model.predict(tx)
            y_pred = np.argmax(y_pred, axis=1)
            y_pred = mode(y_pred)[0]
            pred_label = labels[y_pred]

            # add 1 to accuracy if prediction match the directory name
            if pred_label == dir:
                accuracy[dir] += 1

            # clear df to free up memory
            del df

# calculate acc by dividing the number of correct predictions by total number of files
for dir in os.listdir(data_path):
    accuracy[dir] = accuracy[dir] / len(os.listdir(data_path + dir))

../data/android_data_cleaned/sit/sit_1.csv
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
../data/android_data_cleaned/sit/sit_2.csv
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/sit/sit_3.csv
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/sit/sit_4.csv
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/sit/sit_5.csv
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
../data/android_data_cleaned/sit/sit_6.csv
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
../data/android_data_cleaned/std/std_1.csv
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/std/std_2.csv
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/std/std_3.csv
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/ups/ups_1.csv
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
../data/android_data_cleaned/wlk/wlk_1.csv
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned/wlk/wlk_2.csv
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
../data/android_data_cleaned

In [48]:
print('acc:', accuracy)
for dir in os.listdir(data_path):
    print('file len of', dir + ':', len(os.listdir(data_path + dir)))

acc: {'sit': 0.8333333333333334, 'std': 0.6666666666666666, 'ups': 0.0, 'wlk': 0.25}
file len of sit: 6
file len of std: 3
file len of ups: 1
file len of wlk: 4
